In [ ]:
from transformers import TFAutoModel,AutoTokenizer
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
model=TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
df=pd.read_csv('/content/dataset.csv2.csv')
df.head()

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern


In [ ]:
df['Pattern Category'].value_counts()

Not Dark Pattern    1178
Scarcity             418
Social Proof         312
Urgency              210
Misdirection         195
Obstruction           27
Sneaking              12
Forced Action          4
Name: Pattern Category, dtype: int64

In [ ]:
tokenized_texts = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 512
padded_texts = tf.keras.preprocessing.sequence.pad_sequences(tokenized_texts, maxlen=max_len, padding='post', truncating='post')

attention_masks = (padded_texts != 0).astype(int)

In [ ]:
class_labels = sorted(df['Pattern Category'].unique())
class_label_mapping = {label: idx for idx, label in enumerate(class_labels)}

df['Pattern Category'] = df['Pattern Category'].map(class_label_mapping)
df['Pattern Category'].unique()

array([7, 2, 4, 1, 6, 3, 5, 0])

In [ ]:
train_inputs, val_inputs, train_labels, val_labels, train_masks, val_masks = train_test_split(
    padded_texts, df['Pattern Category'].values, attention_masks, test_size=0.2, random_state=42
)

In [ ]:
def calculate_class_weights(y):
    class_counts = pd.Series(y).value_counts()
    total_samples = len(y)
    class_weights = total_samples / (len(class_counts) * class_counts)
    return class_weights.to_dict()

class_weights = calculate_class_weights(df['Pattern Category'])
class_weights

{2: 0.25,
 4: 0.7045454545454546,
 6: 0.9439102564102564,
 7: 1.4023809523809523,
 1: 1.5102564102564102,
 3: 10.907407407407407,
 5: 24.541666666666668,
 0: 73.625}

In [ ]:
class BERTForClassification(tf.keras.Model):

  def __init__(self, bert_model, num_classes):
    super().__init__()
    self.bert = bert_model
    self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.bert(inputs)[1]
    return self.fc(x)

In [ ]:
clf=BERTForClassification(model, num_classes=8)

In [ ]:
clf.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history=clf.fit(
    train_inputs,
    train_labels,
    epochs=5,  # Adjust as needed
    batch_size=10,
    validation_data=(val_inputs, val_labels),
    class_weight=class_weights,  # Pass class weights here
)

Epoch 1/5
189/189 [==============================] - 242s 1s/step - loss: 0.6511 - accuracy: 0.8376 - val_loss: 0.3247 - val_accuracy: 0.9131
Epoch 2/5
189/189 [==============================] - 253s 1s/step - loss: 0.2791 - accuracy: 0.9071 - val_loss: 0.3282 - val_accuracy: 0.8835
Epoch 3/5
189/189 [==============================] - 253s 1s/step - loss: 0.1719 - accuracy: 0.9390 - val_loss: 0.2524 - val_accuracy: 0.9025
Epoch 4/5
189/189 [==============================] - 254s 1s/step - loss: 0.1169 - accuracy: 0.9496 - val_loss: 0.1998 - val_accuracy: 0.9301
Epoch 5/5
189/189 [==============================] - 253s 1s/step - loss: 0.0784 - accuracy: 0.9639 - val_loss: 0.1945 - val_accuracy: 0.9343


In [ ]:
class_label={idx:label for idx, label in enumerate(class_labels)}
class_label,class_label_mapping

({0: 'Forced Action',
  1: 'Misdirection',
  2: 'Not Dark Pattern',
  3: 'Obstruction',
  4: 'Scarcity',
  5: 'Sneaking',
  6: 'Social Proof',
  7: 'Urgency'},
 {'Forced Action': 0,
  'Misdirection': 1,
  'Not Dark Pattern': 2,
  'Obstruction': 3,
  'Scarcity': 4,
  'Sneaking': 5,
  'Social Proof': 6,
  'Urgency': 7})

In [ ]:
df.head(50)

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,7
1,158,Pillowcases & Shams,0,2
2,108,Write a review,0,2
3,1425,"To start your return, simply click on the foll...",0,2
4,1658,newsletter signup (privacy policy),0,2
5,975,In demand,1,4
6,1469,We cannot guarantee door to door delivery for ...,0,2
7,1570,Hurry! Only 2 left in stock,1,4
8,1617,In Stock only 3 left,1,4
9,108,International Shipping Policy,0,2
